In [1]:
import evalml
from evalml import *
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from agents.tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from agents.data_agent import *
from agents.stock_agent import *
from backtest import *
from bitstamp import *
from model_winner_select import *


from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

step = 1


In [3]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = train_keys,
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list,
                 verbose = False
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [4]:
load_from_disk = True
tec = TecAn(windows = 20, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(3, sourceDataGenerator, load_from_disk, 10)

In [5]:
trainX_raw, trainY_raw = online.load_train_data()


x, y = prepare_train_data(trainX_raw, trainY_raw, step)


print(Counter(y))

Counter({0: 25444, 1: 3356})


In [6]:

trainX_raw, trainY_raw, prices = online.load_val_data("btcusd")
val_x, val_y = prepare_train_data(trainX_raw, trainY_raw, step)

print(Counter(val_y))

Counter({0: 12834, 1: 1567})


In [7]:
from tpot.config import classifier_config_dict
# add FeatureSetSelector into tpot configuration
classifier_config_dict['tpot.builtins.FeatureSetSelector'] = {
    'subset_list': ['https://raw.githubusercontent.com/EpistasisLab/tpot/master/tests/subset_test.csv'],
    'sel_subset': [0,1] # select only one feature set, a list of index of subset in the list above
    #'sel_subset': list(combinations(range(3), 2)) # select two feature sets
}



In [ ]:
%%time

from tpot import TPOTClassifier

model = TPOTClassifier(#generations=30, 
                       #population_size=50,
                       max_time_mins=15*60,
                       scoring='balanced_accuracy', 
                       verbosity=2, 
                       template='Selector-Transformer-Classifier',
                       #config_dict=classifier_config_dict,
                       random_state=1,
                       #config_dict='TPOT sparse',
                       n_jobs=-1)
# perform the search
model.fit(x, y)

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6400410189948851

Generation 2 - Current best internal CV score: 0.6450652282014343

Generation 3 - Current best internal CV score: 0.6450652282014343

Generation 4 - Current best internal CV score: 0.6450652282014343

Generation 5 - Current best internal CV score: 0.6450652282014343

Generation 6 - Current best internal CV score: 0.6450652282014343

Generation 7 - Current best internal CV score: 0.6450652282014343

Generation 8 - Current best internal CV score: 0.6452418752403268

Generation 9 - Current best internal CV score: 0.6452418752403268

Generation 10 - Current best internal CV score: 0.6452418752403268

Generation 11 - Current best internal CV score: 0.6452418752403268

Generation 12 - Current best internal CV score: 0.6452418752403268

Generation 13 - Current best internal CV score: 0.6452966608253616

Generation 14 - Current best internal CV score: 0.6453699066690334


In [ ]:
print(model.score(val_x, val_y))

In [ ]:
cache = CacheProvider(
        currency_list=["btcusd"],
        verbose = False
    )

In [ ]:
val_keys = online.val_keys
for key in val_keys:
    print(key)
    back, metric = eval_model(model = model,
                      hot_load_total = 100,
                      currency = key, 
                      cache = cache,
                      provider = online, step = step, verbose = False)
    back.report()
    print(metric)

In [ ]:
model.export('tpot_boston_pipeline.py')

In [ ]:
automl = AutoMLSearch(X_train=x, y_train=y, problem_type='binary', 
                      max_batches=4,
                      objective='f1',
                      #additional_objectives=['auc', 'f1', 'precision'],
                      #additional_objectives=[ 'precision'],
                      ensembling=False)
automl.search()

In [ ]:
#automl.rankings

In [ ]:
pipeline = automl.best_pipeline

#pipeline.graph()

eval_data(pipeline, val_x, val_y)


In [ ]:
preds = pipeline.predict(val_x)

print(Counter(val_y))
print(Counter(preds))

In [ ]:
evalml.objectives.utils.get_core_objective_names()

In [ ]:
back = test_model(pipeline, "btcusd", online, step, verbose = True)

In [ ]:
back.report()